<a href="https://colab.research.google.com/github/ykitaguchi77/ImageProcessing/blob/master/Numpy_preprocessing_for_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Numpy image processing for Pytorch**



In [1]:
import torch
import os
import IPython
import shutil
import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np




print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Setup complete. Using torch 1.12.1+cu113 CPU
Mounted at /content/drive


In [ ]:
image_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px"

img_list = glob.glob(f"{image_folder}/*")
img = img_list[0]
print(img)

#showimage
# img = "/content/drive/MyDrive/Deep_learning/Face_Images/test_dog.jpg"
img_np = np.array(Image.open(img)).astype(np.float32)
print(img_np)
plt.imshow((img_np).astype(np.uint8))
plt.show()

In [ ]:
#ImageNetでnormalizeする（ImageとNumpyで255倍異なるので補正）
def zscore(x, axis=None):
    # xmean = x.mean(axis=axis, keepdims=True)
    xmean = [n*255 for n in [0.485, 0.456, 0.406]]
    # xstd = np.std(x, axis=axis, keepdims=True)
    xstd = [n*255 for n in [0.229, 0.224, 0.225]]
    zscore = (x-xmean)/xstd
    return zscore

PIL_img = Image.open(img)
PIL_img = PIL_img.resize((224,224), resample=0)

#convert PIL to numpy
img_np = np.array(PIL_img).astype(np.float32)
img_np = zscore(img_np)

print(img_np)

plt.imshow(img_np)
print(img_np)
plt.show()


In [ ]:
#torch.transformsと同じ値になることを確認
from torchvision import models, transforms

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

PIL_img = Image.open(img)
img_tensor = preprocess(PIL_img)
img_np = img_tensor.to("cpu").detach().numpy().transpose(1, 2, 0).copy()
print(img_np)

#**Preprocess multiple images**

In [9]:
orig_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px/aaa.jpg"
os.path.basename(orig_path)


'aaa.jpg'

In [ ]:
from torchvision import models, transforms
import shutil
import os

orig_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px"
dst_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_224px_normalized"

def preprocess(img):
    PIL_img = Image.open(img)
    img_tensor = preprocess(PIL_img)
    img_np = img_tensor.to("cpu").detach().numpy().transpose(1, 2, 0).copy()
    PIL_image = Image.fromarray(img_np)
    print(PIL_image)

if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
os.makedirs(dst_folder) 

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

for path in glob.glob(f"{orig_folder}/*"):
    img = Image.open(path)
    pil_img = preprocess(img)
    print(pil_img)
    #pil_img.save(f"{os.path.basename(path)}")
    print(f"{os.path.basename(path)} saved!")





